In [36]:
import ee
ee.Initialize()
import requests
import os
import zipfile
#RnkUmmi -52.89 71.55 -51.43 71.82

The process to authenticate Earth Engine

In [37]:
#Creates a geometry object with cordinates of RnkUmmi
geometry = ee.Geometry.Rectangle([-52.89, 71.55, -51.43, 71.82])

#variables that Jonathan had
data_product = 'LANDSAT/LE07/C01/T1'
sel_bands = ['B5','B4','B3']
year = 2011
start = ee.Date(f'{year}-07-20')
end = ee.Date(f'{year}-09-10')

#This is a imageCollection object that has all the filters applied
filteredCollection = ee.ImageCollection(data_product).filterBounds(geometry).filterDate(start, end).select(sel_bands).filterMetadata('CLOUD_COVER', 'less_than', 30)




In [38]:


#How many images are in the object
count = filteredCollection.size().getInfo()

print("count is %d"%(count))

#Why does the collection size say 5 when there are only 3 images? returns (Bands) and (Properties)

if(count > 0):
    print("")

#creates a temp folder to download the images to
save_folder =  '_Landsat-7'
os.makedirs(save_folder, exist_ok=True)

#Why is this list filtered?
filtered_list=filteredCollection.toList(filteredCollection.size())

#Make sure the list are inside the geometry object
for i in range(count):
    image = ee.Image(filtered_list.get(i)).clip(geometry)

    #creates the download url for the image
    url = image.getDownloadURL()

    #makes a request to download the image
    r=requests.get(url)

    #names each file 
    filename = f"data_{i}.zip"
    filepath = os.path.join(save_folder, filename)
    print(f"Downloading to: {filepath}")

    try:
        #Download and saves the image
        with open(filepath, 'wb') as f:
            f.write(r.content)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Error failure to download")
    try:
        with zipfile.ZipFile(filepath) as f:
            files = f.namelist()
            f.extractall(save_folder)
        os.remove(filepath)
    except zipfile.BadZipFile:
        print('Skipping - Not a valid ZIP file')
    except Exception as e:
        print('Error extracting')
# image = ee.Image(filtered_list.get(0)).clip(geometry)


print("Downloading complete")
#Downloads the data locally
# name='./'+save_folder+'/'+'data.zip'
# path='./'+save_folder


count is 5

Downloaded data_0.zip
Error extracting
Downloaded data_1.zip
Error extracting
Downloaded data_2.zip
Error extracting
Downloaded data_3.zip
Error extracting
Downloaded data_4.zip
Error extracting
